# Loading Libraries

## General

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import random
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import os
from sklearn.feature_selection import RFECV
import pandas as pd
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    KFold,
)
from sklearn.metrics import f1_score
import numpy as np
from numpy.lib.function_base import average
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    recall_score,
    f1_score,
    precision_score,
    confusion_matrix,
    make_scorer,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    KFold,
)
from sklearn.feature_selection import SelectKBest, SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import importlib
import os
from sklearn.feature_selection import (
    SelectKBest,
    RFE,
    mutual_info_regression,
    f_regression,
    mutual_info_classif,
)
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.inspection import permutation_importance
import xgboost as xgb
import random
import pickle
import openpyxl
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import pickle


## Local

In [ ]:
# Setting path to the initial folder
os.chdir("C:\\Users\\Marieke\\GitHub\\Typhoon_IBF_Rice_Damage_Model")
cdir = os.getcwd()

import importlib

from IBF_typhoon_model.models.binary_classification.xgb_binary import xgb_binary_features, xgb_binary_performance
from IBF_typhoon_model.models.binary_classification.rf_binary import rf_binary_features, rf_binary_performance

from IBF_typhoon_model.models.multiclass_classification.rf_multi import rf_multi_features, rf_multi_performance

from IBF_typhoon_model.models.regression.rf_regression import rf_regression_features, rf_regression_performance


from IBF_typhoon_model.models.utility_functions.splitting_train_test import splitting_train_test
from IBF_typhoon_model.models.utility_functions.determine_class import determine_class

# Notebook to obtain the model performance

## Loading the Dataset

In [3]:
# Input data
name = "IBF_typhoon_model\\data\\restricted_data\\combined_input_data\\input_data_02.xlsx"
path = os.path.join(cdir, name)
df = pd.read_excel(path, engine="openpyxl")

# Typhoon overview
file_name = "IBF_typhoon_model\\data\\restricted_data\\data_overview.xlsx"
path = os.path.join(cdir, file_name)
df_typh_overview = pd.read_excel(path, sheet_name="typhoon_overview", engine="openpyxl")

C:\Users\Marieke\anaconda3\envs\Rice_Field_Damage_Philippines\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Selecting the features to be used
features = [
    "rice_area",
    "mean_slope",
    "mean_elevation_m",
    "ruggedness_stdev",
    "mean_ruggedness",
    "slope_stdev",
    "area_km2",
    "poverty_perc",
    "with_coast",
    "coast_length",
    "perimeter",
    "glat",
    "glon",
    "coast_peri_ratio",
    "rainfall_sum",
    "rainfall_max",
    "dis_track_min",
    "vmax_sust",
]

# Binary Classification

In [30]:
# Setting the general input variables
threshold = 0.3
df["class_value_binary"] = [1 if df["perc_loss"][i] > threshold else 0 for i in range(len(df))]

# Setting for feature selection on full data set
X = df[features]
y = df['class_value_binary']
y = y.astype(int)

# Setting the train and the test sets for obtaining performance estimate
df_train_list, df_test_list = splitting_train_test(df)

## Random Forest

Number of selected features RF Binary: 15

Selected features RF Binary: ['rice_area', 'mean_elevation_m', 'ruggedness_stdev', 'mean_ruggedness', 'slope_stdev', 'area_km2', 'poverty_perc', 'perimeter', 'glat', 'glon', 'coast_peri_ratio', 'rainfall_sum', 'rainfall_max', 'dis_track_min', 'vmax_sust']

Selected Parameters RF Binary {'estimator__max_depth': None, 'estimator__min_samples_leaf': 3, 'estimator__min_samples_split': 10, 'estimator__n_estimators': 100}

In [6]:
# Setting the random forest search grid
rf_search_space = [{
        "estimator__n_estimators": [50, 100, 500],
        "estimator__max_depth": [20, None],
        "estimator__min_samples_split": [2, 8, 10],
        "estimator__min_samples_leaf": [1, 3, 5],
    }]

# Obtaining the selected features based on the full dataset
selected_features_rf_binary,  selected_params_rf_binary_full = rf_binary_features(
    X=X,
    y=y,
    features=features,
    search_space=rf_search_space,
    cv_splits=5,
    class_weight='balanced',
    min_features_to_select=1,
    GS_score='f1',
    GS_randomized=False,
    GS_n_iter=10,
)

print(f"Number of selected features RF Binary: {len(selected_features_rf_binary)}")
print(f"Selected features RF Binary: {selected_features_rf_binary}")
print(f"Selected Parameters RF Binary {selected_params_rf_binary_full}")

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5; 1/54] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 1/5; 1/54] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=0.998, test=0.559) total time=  27.0s
[CV 2/5; 1/54] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 2/5; 1/54] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=0.999, test=0.603) total time=  24.4s
[CV 3/5; 1/54] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 3/5; 1/54] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=0.997, 

In [7]:
# # Setting the selected features for RF --> based on outcome in previous cell
# selected_features_rf_binary = [
#     'rice_area', 
#     'mean_slope', 
#     'mean_elevation_m', 
#     'ruggedness_stdev', 
#     'mean_ruggedness', 
#     'slope_stdev', 
#     'area_km2', 
#     'poverty_perc', 
#     'with_coast', 
#     'coast_length', 
#     'perimeter', 
#     'glat', 
#     'glon', 
#     'coast_peri_ratio', 
#     'rainfall_sum', 
#     'rainfall_max', 
#     'dis_track_min', 
#     'vmax_sust'
# ]

In [9]:
# Setting the random forest search grid
rf_search_space = [{
        "rf__n_estimators": [50, 100, 500],
        "rf__max_depth": [20, None],
        "rf__min_samples_split": [2, 8, 10],
        "rf__min_samples_leaf": [1, 3, 5],
    }]

# Obtaining the performance estimate
df_predicted_rf_binary, selected_params_rf_binary = rf_binary_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_rf_binary,
    search_space=rf_search_space,
    stratK=True,
    cv_splits=5,
    class_weight='balanced',
    GS_score='f1',
    GS_randomized=False,
    GS_n_iter=10,
)

Running for 1 out of a total of 5
Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5; 1/54] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 1/5; 1/54] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=0.999, test=0.529) total time=   0.3s
[CV 2/5; 1/54] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 2/5; 1/54] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=0.997, test=0.511) total time=   0.3s
[CV 3/5; 1/54] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 3/5; 1/54] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=0.996, test=0.561) total time=   0.3s
[CV 4/5; 1/54] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimator

In [24]:
file_name = "IBF_typhoon_model\\models\\output\\selected_params_rf_binary.p"
path = os.path.join(cdir, file_name)
pickle.dump(selected_params_rf_binary, open(path, "wb" ))

## XGBoost

In [32]:
# Setting the XGBoost search grid for full dataset
xgb_search_space = [{
        "estimator__learning_rate": [0.1, 0.5, 1],
        "estimator__gamma": [0.1, 0.5, 2],
        "estimator__max_depth": [6, 8],
        "estimator__reg_lambda": [0.001, 0.01, 0.1, 1],
        "estimator__n_estimators": [100, 200],
        "estimator__colsample_bytree": [0.5, 0.7, 1],
    }]

# Obtaining the selected features based on the full dataset
selected_features_xgb_binary, selected_params_xgb_binary_full = xgb_binary_features(
    X=X, 
    y=y, 
    features=features, 
    search_space=xgb_search_space, 
    objective="binary:hinge", 
    cv_splits=5, 
    min_features_to_select=1, 
    GS_score='f1', 
    GS_n_iter=1,
    GS_randomized=False
)

print(f"Number of selected features XGBoost Binary {len(selected_features_xgb_binary)}")
print(f"Selected features XGBoost Binary: {selected_features_xgb_binary}")
print(f"Selected parameters XGBoost Binary: {selected_params_xgb_binary_full}")

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5; 1/432] START estimator__colsample_bytree=0.5, estimator__gamma=0.1, estimator__learning_rate=0.1, estimator__max_depth=6, estimator__n_estimators=100, estimator__reg_lambda=0.001
[CV 1/5; 1/432] END estimator__colsample_bytree=0.5, estimator__gamma=0.1, estimator__learning_rate=0.1, estimator__max_depth=6, estimator__n_estimators=100, estimator__reg_lambda=0.001;, score=(train=0.667, test=0.543) total time=  42.1s
[CV 2/5; 1/432] START estimator__colsample_bytree=0.5, estimator__gamma=0.1, estimator__learning_rate=0.1, estimator__max_depth=6, estimator__n_estimators=100, estimator__reg_lambda=0.001
[CV 2/5; 1/432] END estimator__colsample_bytree=0.5, estimator__gamma=0.1, estimator__learning_rate=0.1, estimator__max_depth=6, estimator__n_estimators=100, estimator__reg_lambda=0.001;, score=(train=0.663, test=0.556) total time=  47.9s
[CV 3/5; 1/432] START estimator__colsample_bytree=0.5, estimator__gamma=0.1, estim

In [ ]:
# # Setting the selected features for XGB --> based on outcome previous cell
# selected_features_xgb = [
#     'rice_area', 
#     'mean_slope', 
#     'mean_elevation_m', 
#     'ruggedness_stdev', 
#     'mean_ruggedness', 
#     'slope_stdev', 
#     'area_km2', 
#     'poverty_perc', 
#     'with_coast', 
#     'coast_length', 
#     'perimeter', 
#     'glat', 
#     'glon', 
#     'coast_peri_ratio', 
#     'rainfall_sum', 
#     'rainfall_max', 
#     'dis_track_min', 
#     'vmax_sust'
# ]

In [11]:
# Setting the XGBoost search grid
xgb_search_space = [{
        "xgb__learning_rate": [0.1, 0.3],
        "xgb__gamma": [0.1, 0.5, 2],
        "xgb__max_depth": [6, 8],
        "xgb__reg_lambda": [0.01, 0.1, 1],
        "xgb__n_estimators": [100, 150],
        "xgb__colsample_bytree": [0.5, 0.7 ],
    }]

# Obtaining the performance estimate
df_predicted_xgb_binary, selected_params_xgb_binary = xgb_binary_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_xgb_binary,
    search_space=xgb_search_space,
    stratK=True,
    cv_splits=5,
    objective="binary:hinge",
    GS_score='f1',
    GS_randomized=False,
    GS_n_iter=10,
)

Running for 1 out of a total of 5
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5; 1/144] START xgb__colsample_bytree=0.5, xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.01
[CV 1/5; 1/144] END xgb__colsample_bytree=0.5, xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.01;, score=(train=0.762, test=0.587) total time=   0.3s
[CV 2/5; 1/144] START xgb__colsample_bytree=0.5, xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.01
[CV 2/5; 1/144] END xgb__colsample_bytree=0.5, xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.01;, score=(train=0.782, test=0.555) total time=   0.4s
[CV 3/5; 1/144] START xgb__colsample_bytree=0.5, xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.01
[CV 3/5; 1/144] END xgb__colsample_bytree=0.5,

## Benchmark

In [ ]:
def unweighted_random(y_train, y_test):
    options = y_train.value_counts(normalize=True)
    y_pred = random.choices(population=list(options.index), k=len(y_test))
    return y_pred


def weighted_random(y_train, y_test):
    options = y_train.value_counts()
    y_pred = random.choices(
        population=list(options.index), weights=list(options.values), k=len(y_test)
    )
    return y_pred

In [ ]:
# Random unweighted predictions
df_predicted = pd.DataFrame(columns=["year", "actual", "predicted"])
for i in range(len(df_train_list)):

    train = df_train_list[i]
    test = df_test_list[i]

    y_train = train["class_value_binary"]
    y_test = test["class_value_binary"]

    y_pred_test = unweighted_random(y_train, y_test)
    df_predicted_temp = pd.DataFrame(
        {"year": test["year"], "actual": y_test, "predicted": y_pred_test}
    )

    df_predicted = pd.concat([df_predicted, df_predicted_temp])

f1_score_binary_random = f1_score(list(df_predicted['actual']), list(df_predicted['predicted']))

In [ ]:
# Random Weighted Predictions
df_predicted = pd.DataFrame(columns=["year", "actual", "predicted"])
for i in range(len(df_train_list)):

    train = df_train_list[i]
    test = df_test_list[i]

    y_train = train["class_value_binary"]
    y_test = test["class_value_binary"]

    y_pred_test = weighted_random(y_train, y_test)
    df_predicted_temp = pd.DataFrame(
        {"year": test["year"], "actual": y_test, "predicted": y_pred_test}
    )

    df_predicted = pd.concat([df_predicted, df_predicted_temp])

f1_score_binary_weighted_random = f1_score(list(df_predicted['actual']), list(df_predicted['predicted']))

# Multiclass Classification

In [10]:
# Setting class value
# Set final boundary slightly over 1 so 1's are included as well
classes = {"0": [0, 0.3], "1": [0.3, 0.8], "2": [0.8, 1.1]}
df['class_value_multi'] = df['perc_loss'].apply(lambda x: determine_class(x, classes=classes))

# Setting for feature seleciton on full data set
X = df[features]
y = df['class_value_multi']

# Setting train and test set for obtaining performance estimate
df_train_list, df_test_list = splitting_train_test(df)

## Random Forest

Number of features selected in RF multiclass: 12
Selected features RF multiclass: ['rice_area', 'mean_elevation_m', 'mean_ruggedness', 'area_km2', 'poverty_perc', 'perimeter', 'glat', 'glon', 'rainfall_sum', 'rainfall_max', 'dis_track_min', 'vmax_sust']
Selected Parameters in RF multiclass: {'estimator__max_depth': 20, 'estimator__min_samples_leaf': 5, 'estimator__min_samples_split': 10, 'estimator__n_estimators': 50}

In [11]:
# Setting the random forest search grid
rf_search_space = [{
        "estimator__n_estimators": [50, 100],
        "estimator__max_depth": [20, None],
        "estimator__min_samples_split": [2, 8, 10],
        "estimator__min_samples_leaf": [1, 3, 5],
    }]

selected_features_rf_multi, selected_params_rf_multi_full = rf_multi_features(
    X=X,
    y=y,
    features=features,
    search_space=rf_search_space,
    cv_splits=5,
    class_weight='balanced',
    min_features_to_select=1,
    GS_score='f1_macro',
    GS_randomized=False,
    GS_n_iter=10,
)

print(f"Number of features selected in RF multiclass: {len(selected_features_rf_multi)}")
print(f"Selected features RF multiclass: {selected_features_rf_multi}")
print(f"Selected Parameters in RF multiclass: {selected_params_rf_multi_full}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 1/5; 1/36] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=1.000, test=0.496) total time=  28.3s
[CV 2/5; 1/36] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 2/5; 1/36] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=1.000, test=0.477) total time=  24.0s
[CV 3/5; 1/36] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 3/5; 1/36] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=0.999, 

In [ ]:
# # Setting the selected features for RF --> based on output previous cell
# selected_features_rf_multi = [
#     'rice_area', 
#     'mean_slope', 
#     'mean_elevation_m', 
#     'ruggedness_stdev', 
#     'mean_ruggedness', 
#     'slope_stdev', 
#     'area_km2', 
#     'poverty_perc', 
#     'with_coast', 
#     'coast_length', 
#     'perimeter', 
#     'glat', 
#     'glon', 
#     'coast_peri_ratio', 
#     'rainfall_sum', 
#     'rainfall_max', 
#     'dis_track_min', 
#     'vmax_sust'
# ]

In [13]:
# Obtain the performance estimate
rf_search_space = [{
        "rf__n_estimators": [50, 100],
        "rf__max_depth": [20, None],
        "rf__min_samples_split": [2, 8, 10],
        "rf__min_samples_leaf": [1, 3, 5],
    }]

df_predicted_rf_multi, selected_params_rf_multi = rf_multi_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_rf_multi,
    search_space=rf_search_space,
    stratK=True,
    cv_splits=5,
    class_weight='balanced',
    GS_score='f1_macro',
    GS_randomized=False,
    GS_n_iter=10,
)

file_name = "IBF_typhoon_model\\models\\output\\selected_params_rf_multi.p"
path = os.path.join(cdir, file_name)
pickle.dump(selected_params_rf_multi, open(path, "wb" ))

file_name = "IBF_typhoon_model\\models\\output\\df_predicted_rf_multi.csv"
path = os.path.join(cdir, file_name)
df_predicted_rf_multi.to_csv(path)

C:\Users\Marieke\anaconda3\envs\Rice_Field_Damage_Philippines\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Running for 1 out of a total of 5
Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 1/5; 1/36] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=1.000, test=0.499) total time=   0.3s
[CV 2/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 2/5; 1/36] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=0.999, test=0.479) total time=   0.3s
[CV 3/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 3/5; 1/36] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=0.999, test=0.472) total time=   0.3s
[CV 4/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimator

C:\Users\Marieke\anaconda3\envs\Rice_Field_Damage_Philippines\lib\site-packages\ipykernel_launcher.py:24: ResourceWarning: unclosed file <_io.BufferedWriter name='C:\\Users\\Marieke\\GitHub\\Typhoon_IBF_Rice_Damage_Model\\IBF_typhoon_model\\models\\output\\selected_params_rf_multi.p'>


In [ ]:
file_name = "IBF_typhoon_model\\models\\output\\selected_params_rf_multi.p"
path = os.path.join(cdir, file_name)
pickle.dump(selected_params_rf_multi, open(path, "wb" ))

file_name = "IBF_typhoon_model\\models\\output\\df_predicted_rf_multi.csv"
path = os.path.join(cdir, file_name)
df_predicted_rf_multi.to_csv(path)

## XGBoost

In [ ]:
# Setting the XGBoost search grid

#TODO Continue from here
xgb_search_space = [{
        "estimator__learning_rate": [0.1, 0.5, 1]
        "estimator__gamma": [0.1, 0.5, 2],
        "estimator__max_depth": [6, 8],
        "estimator__reg_lambda": [0.001, 0.01, 0.1, 1],
        "estimator__n_estimators": [100, 200],
        "estimator__colsample_bytree": [0.5, 0.7, 1],
    }]

selected_features_xgb = xgb_multi_features(
    X=X,
    y=y,
    features=selected_features_xgb,
    num_class=len(classes),
    search_space=xgb_search_space,
    objective="multi:softmax",
    cv_splits=5,
    min_features_to_select=7,
    GS_score='f1_macro',
    GS_randomized=True,
    GS_n_iter=10,
)

print(f"Selected features XGBoost: {selected_features_xgb}")

## Benchmark

In [ ]:
def unweighted_random(y_train, y_test):
    options = y_train.value_counts(normalize=True)
    y_pred = random.choices(population=list(options.index), k=len(y_test))
    return y_pred


def weighted_random(y_train, y_test):
    options = y_train.value_counts()
    y_pred = random.choices(
        population=list(options.index), weights=list(options.values), k=len(y_test)
    )
    return y_pred

In [ ]:
# Random unweighted predictions
df_predicted = pd.DataFrame(columns=["year", "actual", "predicted"])
for i in range(len(df_train_list)):

    train = df_train_list[i]
    test = df_test_list[i]

    y_train = train["class_value_multi"]
    y_test = test["class_value_multi"]

    y_pred_test = unweighted_random(y_train, y_test)
    df_predicted_temp = pd.DataFrame(
        {"year": test["year"], "actual": y_test, "predicted": y_pred_test}
    )

    df_predicted = pd.concat([df_predicted, df_predicted_temp])

f1_score_multi_random = f1_score(list(df_predicted['actual']), list(df_predicted['predicted']), average='macro')

In [ ]:
# Random Weighted Predictions
df_predicted = pd.DataFrame(columns=["year", "actual", "predicted"])
for i in range(len(df_train_list)):

    train = df_train_list[i]
    test = df_test_list[i]

    y_train = train["class_value_multi"]
    y_test = test["class_value_multi"]

    y_pred_test = weighted_random(y_train, y_test)
    df_predicted_temp = pd.DataFrame(
        {"year": test["year"], "actual": y_test, "predicted": y_pred_test}
    )

    df_predicted = pd.concat([df_predicted, df_predicted_temp])

f1_score_multi_weighted_random = f1_score(list(df_predicted['actual']), list(df_predicted['predicted']), average='macro')

## Obtain performance estimate

In [135]:
# Setting the train and the test sets


<IPython.core.display.Javascript object>

In [ ]:
# Setting the selected features for XGB
selected_features_xgb = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [ ]:
# Setting the XGBoost search grid
xgb_search_space = [{
        "xgb__learning_rate": [0.1, 0.5, 1],
        "xgb__gamma": [0.1, 0.5, 2],
        "xgb__max_depth": [6, 8],
        "xgb__reg_lambda": [0.001, 0.01, 0.1, 1],
        "xgb__n_estimators": [100, 200],
        "xgb__colsample_bytree": [0.5, 0.7, 1],
    }]

df_predicted_xgb = xgb_multi_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    num_class=len(classes),
    features=selected_features_xgb,
    search_space=xgb_search_space,
    stratK=True,
    cv_splits=5,
    objective="multi:softmax",
    GS_score='f1_macro',
    GS_randomized=True,
    GS_n_iter=10,
)

# Regression

In [6]:
# Full dataset for feature selection
X = df[features]
y = df['perc_loss']

# Setting the train and the test sets for obtaining performance estimate
df_train_list, df_test_list = splitting_train_test(df)

## Random Forest

Number of selected features RF Regression 12
Selected features RF Regression: ['rice_area', 'mean_elevation_m', 'mean_ruggedness', 'poverty_perc', 'perimeter', 'glat', 'glon', 'coast_peri_ratio', 'rainfall_sum', 'rainfall_max', 'dis_track_min', 'vmax_sust']
Selected Parameters RF Regression: {'estimator__max_depth': None, 'estimator__min_samples_leaf': 3, 'estimator__min_samples_split': 8, 'estimator__n_estimators': 100}

In [7]:
#%% Setting input varialbes
rf_search_space = [{
        "estimator__n_estimators": [50, 100],
        "estimator__max_depth": [20, None],
        "estimator__min_samples_split": [2, 8, 10],
        "estimator__min_samples_leaf": [1, 3, 5],
    }]

selected_features_rf_regression, selected_params_rf_regression_full = rf_regression_features(
    X=X,
    y=y,
    features=features,
    search_space=rf_search_space,
    min_features_to_select=1,
    cv_splits=5,
    GS_score="neg_root_mean_squared_error",
    GS_randomized=False,
    GS_n_iter=10,
)

print(f"Number of selected features RF Regression {len(selected_features_rf_regression)}")
print(f"Selected features RF Regression: {selected_features_rf_regression}")
print(f"Selected Parameters RF Regression: {selected_params_rf_regression_full}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 1/5; 1/36] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=-0.100, test=-0.252) total time= 1.7min
[CV 2/5; 1/36] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 2/5; 1/36] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=-0.100, test=-0.263) total time= 1.5min
[CV 3/5; 1/36] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50
[CV 3/5; 1/36] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50;, score=(train=-0.

In [ ]:
# Based on output previous cell
selected_features_rf_regression = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [15]:
#%% Setting input varialbes
rf_search_space = [{
        "rf__n_estimators": [50, 100],
        "rf__max_depth": [20, None],
        "rf__min_samples_split": [2, 8, 10],
        "rf__min_samples_leaf": [1, 3, 5],
    }]

df_predicted_rf_regression, selected_params_rf_regression = rf_regression_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_rf_regression,
    search_space=rf_search_space,
    cv_splits=5,
    GS_score="neg_root_mean_squared_error",
    GS_randomized=False,
    GS_n_iter=10,
)

file_name = "IBF_typhoon_model\\models\\output\\selected_params_rf_regression.p"
path = os.path.join(cdir, file_name)
pickle.dump(selected_params_rf_regression, open(path, "wb" ))

file_name = "IBF_typhoon_model\\models\\output\\df_predicted_rf_regression.csv"
path = os.path.join(cdir, file_name)
df_predicted_rf_multi.to_csv(path)

C:\Users\Marieke\anaconda3\envs\Rice_Field_Damage_Philippines\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Running for 1 out of a total of 5
Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 1/5; 1/36] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=-0.098, test=-0.252) total time=   1.5s
[CV 2/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 2/5; 1/36] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=-0.098, test=-0.262) total time=   1.6s
[CV 3/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50
[CV 3/5; 1/36] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50;, score=(train=-0.099, test=-0.252) total time=   1.5s
[CV 4/5; 1/36] START rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_est

C:\Users\Marieke\anaconda3\envs\Rice_Field_Damage_Philippines\lib\site-packages\ipykernel_launcher.py:22: ResourceWarning: unclosed file <_io.BufferedWriter name='C:\\Users\\Marieke\\GitHub\\Typhoon_IBF_Rice_Damage_Model\\IBF_typhoon_model\\models\\output\\selected_params_rf_regression.p'>


In [16]:
file_name = "IBF_typhoon_model\\models\\output\\selected_params_rf_regression.p"
path = os.path.join(cdir, file_name)
pickle.dump(selected_params_rf_regression, open(path, "wb" ))

file_name = "IBF_typhoon_model\\models\\output\\df_predicted_rf_regression.csv"
path = os.path.join(cdir, file_name)
df_predicted_rf_multi.to_csv(path)

C:\Users\Marieke\anaconda3\envs\Rice_Field_Damage_Philippines\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\Marieke\anaconda3\envs\Rice_Field_Damage_Philippines\lib\site-packages\ipykernel_launcher.py:3: ResourceWarning: unclosed file <_io.BufferedWriter name='C:\\Users\\Marieke\\GitHub\\Typhoon_IBF_Rice_Damage_Model\\IBF_typhoon_model\\models\\output\\selected_params_rf_regression.p'>
  This is separate from the ipykernel package so we can avoid doing imports until


## Feature selections

In [ ]:
xgb_search_space = [{
        "estimator__learning_rate": [0.1, 0.5, 1],
        "estimator__gamma": [0.1, 0.5, 2],
        "estimator__max_depth": [6, 8],
        "estimator__reg_lambda": [0.001, 0.01, 0.1, 1],
        "estimator__n_estimators": [100, 200],
        "estimator__colsample_bytree": [0.5, 0.7, 1],
    }]

selected_features_xgb = xgb_regression_features(
    X=X,
    y=y,
    features=features,
    search_space=xgb_search_space,
    min_features_to_select=7,
    cv_splits=5,
    GS_score="neg_root_mean_squared_error",
    objective='"reg:squarederror"',
    GS_randomized=True,
    GS_n_iter=10,
)

print(f"Selected features RF: {selected_features_xgb}")

## Obtaining performance Estimate

In [ ]:
# Setting the selected features for rf
selected_features_rf = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [ ]:
#%% Setting input varialbes
rf_search_space = [{
        "rf__n_estimators": [50, 100, 500],
        "rf__max_depth": [20, None],
        "rf__min_samples_split": [2, 8, 10],
        "rf__min_samples_leaf": [1, 3, 5],
    }]

df_predicted_rf = rf_regression_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_rf,
    search_space=rf_search_space,
    cv_splits=5,
    GS_score="neg_root_mean_squared_error",
    GS_randomized=True,
    GS_n_iter=10,
)

In [ ]:
# Setting the selected features for XGB
selected_features_xgb = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [ ]:
xgb_search_space = [{
        "xgb__learning_rate": [0.1, 0.5, 1],
        "xgb__gamma": [0.1, 0.5, 2],
        "xgb__max_depth": [6, 8],
        "xgb__reg_lambda": [0.001, 0.01, 0.1, 1],
        "xgb__n_estimators": [100, 200],
        "xgb__colsample_bytree": [0.5, 0.7, 1],
    }]

df_predicted_xgb = xgb_regression_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=features,
    search_space=xgb_search_space,
    cv_splits=5,
    objective="reg:squarederror",
    GS_score="neg_root_mean_squared_error",
    GS_randomized=True,
    GS_n_iter=10
)